# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count

In [10]:
# create a Spark session
spark = SparkSession.builder.appName("Sparkify Project").getOrCreate()

/opt/venv/lib/python3.10/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
JAVA_HOME is not set


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [ ]:
# Load the mini dataset
file_path = "mini_sparkify_event_data.json"
df = spark.read.json(file_path)
df.show(5)  # Take a look at the first few rows of the data


# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [ ]:
# Check for missing values
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

# Filter out records with null or empty userId and sessionId
df_cleaned = df.filter((col('userId') != '') & (col('sessionId').isNotNull()))


In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

# Define a user-defined function (UDF) to label churn events
def is_churn(event):
    if event == "Cancellation Confirmation":
        return 1
    return 0

churn_udf = udf(is_churn, IntegerType())
df_cleaned = df_cleaned.withColumn("Churn", churn_udf(col("page")))


In [ ]:
# Count distinct users who churned
df_churned = df_cleaned.filter(col("Churn") == 1).select("userId").distinct()
df_stayed = df_cleaned.filter(col("Churn") == 0).select("userId").distinct()

# Aggregate example: Songs played per user
df_cleaned.groupBy("userId").agg(count("song").alias("songs_played")).show(5)


# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [ ]:
# Example feature: number of thumbs up per user
df_features = df_cleaned.groupBy("userId").agg(count(when(col("page") == "Thumbs Up", True)).alias("thumbs_up"))


# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [ ]:
# Split the dataset into train, validation, and test sets
train, test = df_cleaned.randomSplit([0.8, 0.2], seed=42)
train, validation = train.randomSplit([0.8, 0.2], seed=42)


In [ ]:
from pyspark.ml.feature import VectorAssembler

# Select features to include in the model
feature_columns = ["thumbs_up", "songs_played", "other_feature_columns"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Transform the dataset
train = assembler.transform(train)
validation = assembler.transform(validation)
test = assembler.transform(test)


In [ ]:
from pyspark.ml.classification import LogisticRegression

# Create a logistic regression model
lr = LogisticRegression(labelCol="Churn", featuresCol="features")

# Train the model
lr_model = lr.fit(train)

# Evaluate the model on the validation set
predictions = lr_model.transform(validation)


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Evaluate F1 Score
evaluator = MulticlassClassificationEvaluator(labelCol="Churn", predictionCol="prediction", metricName="f1")
f1_score = evaluator.evaluate(predictions)
print(f"F1 Score: {f1_score}")


In [ ]:
from pyspark.ml.classification import RandomForestClassifier

# Create a Random Forest classifier
rf = RandomForestClassifier(labelCol="Churn", featuresCol="features", numTrees=100)

# Train the model
rf_model = rf.fit(train)

# Evaluate on validation set
rf_predictions = rf_model.transform(validation)
f1_rf = evaluator.evaluate(rf_predictions)
print(f"F1 Score (Random Forest): {f1_rf}")


In [ ]:
# Test the winning model on the test set
test_predictions = best_model.transform(test)
f1_test = evaluator.evaluate(test_predictions)
print(f"F1 Score on Test Set: {f1_test}")


# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.